In [ ]:
%run ../commons/imports_legal.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
create_database('blp_dataset')
create_table('products_tmp')

# == Files config ==
FILE_FOLDER = r"D:\legal_data\flipkart_com-ecommerce_sample.csv"
fdata=pd.read_csv(Path(FILE_FOLDER))
IMG_FOLDER_DEST = r"D:\images\flipkart"

In [ ]:
fdata.columns

In [ ]:
# Deleting unwanted columns
del_list = ['uniq_id', 'crawl_timestamp', 'product_category_tree', 'pid', 'discounted_price', 
            'is_FK_Advantage_product', 'overall_rating', 'product_specifications']

fdata = fdata.drop(del_list, axis = 1)

In [ ]:
def insert_product(row):
    query = "INSERT INTO products_tmp (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
    values = (row['product_name'], row['description'], "Flipkart", row['brand'], "f")

    cursor.execute(query, values)
    conn.commit()

    return cursor.lastrowid

In [ ]:
def update_image(product_id, value):
    query = "UPDATE products_tmp SET has_image = %s WHERE id = %s"
    cursor.execute(query, (value, product_id))
    conn.commit()

In [ ]:
def get_ext(url):
    filename, file_extension = os.path.splitext(url)
    
    return file_extension

In [ ]:
for index, row in fdata.iterrows():
    img_list = row['image']
    
    if row['product_name'] == "NaN" or type(row['product_name']) is not str: row['product_name'] = ""
    if row['description'] == "NaN" or type(row['description']) is not str: row['description'] = ""
    if row['brand'] == "NaN" or type(row['brand']) is not str: row['brand'] = ""
        
    if row['product_name'] is not "" and row['description'] is not "":
        # Insert product into database
        product_id = insert_product(row)

        if img_list and img_list is not None and type(row['product_name']) is str:
            try:
                img_list = img_list.strip('][').split(', ')        
                img_index = 0

                for img in img_list:
                    try:
                        update_image(product_id, "no_image")
                        img = img.strip('\'"')
                        folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}{get_ext(img)}")

                        if img_index > 0:
                            folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}_{img_index}{get_ext(img)}")

                        try:
                            file = urllib.request.urlretrieve(img, folder_dest)
                            img_index = img_index + 1
                        except:
                            print(f"Product {product_id}: Error retrieving image ({img})")

                        if img_index == 0 : update_image(product_id, "no_image")
                    except:
                        print("First float problem")
                        print(img)
            except:
                print("Float problem")
                print(img)

                update_image(product_id, "no_image")